### Overview

Tested again on July 31st and August 1st


This notebook contains various scripts to load data into tables on a local DuckDB database. <br>
- proteins are loaded into W2V_PROTEIN
- pfam entries are loaded into W2V_TOKEN
- disorder regions are also loaded into W2V_TOKEN

The tables are created at the time the data is loaded - so see the appropariate cells for the table definition.

Indexes are applied after the data is loaded.


DuckDB is very easy to install on a mac and can load tab-delimited files extremely quickly.
To recreate this environment, you just need to install DuckDB and then set the db_string at the top of this file
to the location where you wish the database file to be stored


## SETUP AND TEST

In [2]:
import duckdb
import time
#
# TODO - SET THIS STRING TO WHERE YOU WANT THE DB TO STORE ITS DATA
#
db_string = "/Users/patrick/dev/ucl/comp0158_mscproject/database/w2v_20240731_test.db"


Test the DB works OK

In [3]:
# CREATE A TABLE
#con = duckdb.connect(database=':memory:')
con = duckdb.connect(database=db_string)  
duckdb.sql("\
    CREATE TABLE TEST (\
        ID VARCHAR,\
    )")
con.close()

In [4]:
# DESCRIBE
con = duckdb.connect(database=db_string)
res = duckdb.sql("DESCRIBE TEST")
print(res)
con.close()

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



In [17]:
# DROP
con = duckdb.connect(database=db_string)  
duckdb.sql("DROP TABLE TEST")
con.close()

## DATA PREPARATION

### LOAD PROTEIN TrEMBL : INTO W2V_PROTEIN

I initialy used TrEMBL to create the corpus as the UniRef100 extract was too large and kept breaking my Macbook!
I subsequently found that it's possible to download only the eukaryortic UniRef100 proteins from the uniprot website. For this, it's necessary to filter on tax id 2759 and also select 100% completion(?). Note that it takes between 12 and 16 hours for Uniprot to prepare the extract for download, but it's worth it as it contains the taxonomy details as well.

In [ ]:
# load protein file into protein table
# 20 July 2024 - This took 12.9s to load uniprotkb-2759_78494531.dat (78M proteins)
# 31 July testing again to check code works
con = duckdb.connect(database=db_string)           
con.execute("CREATE TABLE W2V_PROTEIN AS SELECT * FROM read_csv_auto('/Volumes/My Passport/data/protein/dat/uniprotkb-2759_78494531.dat', columns={'uniprot_id' :'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [11]:
# This should output that there are 78,494,529 items
con = duckdb.connect(database=db_string)           
protein_count = con.execute("SELECT COUNT(*) FROM W2V_PROTEIN").fetchall()
print(protein_count)
con.close()

[(78494529,)]


In [9]:
con = duckdb.connect(database=db_string)           
#con.execute("DROP TABLE W2V_PROTEIN")
con.close()

In [12]:
# create an index (after loading the data)
con = duckdb.connect(database=db_string)   
con.execute("CREATE INDEX UNIP_IDX ON W2V_PROTEIN(UNIPROT_ID)")
print('index created')
con.close()

index created


In [10]:
con = duckdb.connect(database=db_string)      

# SELECT FROM LIST OF IDS - REALLY SLOW
#list = ['A0A010R6E0', 'A0A010RP22']
#entries = con.execute("SELECT * FROM PFAM_TOKEN WHERE column0 IN (SELECT UNNEST(?))", [list]).fetchall()

res = con.execute("SELECT * FROM W2V_PROTEIN WHERE UNIPROT_ID = (?)", ['A0A010PZU8']).fetchall()

print(res)
con.close()

[('A0A010PZU8', 1, 1389)]


### LOAD PROTEIN - UNIREF :

UniRef100 - All Eukaryotic Proteins - including Taxonomy Details

In [4]:
# load protein file into protein table
# 05 Aug 2024 - Took 1min 4.5s to load 95,272,305 items
#
# [counter, uniprot_id, len, start, end, n_members, tax_id, tax_name]
#
con = duckdb.connect(database=db_string)           
con.execute("CREATE TABLE W2V_PROTEIN_UREF100_E AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/protein/uniref100only_2759-95272305_20240805.dat', columns={'counter' : UINTEGER, 'uniprot_id' :VARCHAR, 'length': USMALLINT, 'start': USMALLINT, 'end': USMALLINT, 'n_members': USMALLINT, 'tax_id' :UINTEGER, 'tax_name' : VARCHAR})")
con.close()

In [3]:
# This should output that there are 95,272,305 items
con = duckdb.connect(database=db_string)           
#res = con.execute("DROP TABLE W2V_PROTEIN_UREF100_E").fetchall()
con.close()

[]


In [9]:
# This should output that there are 95,272,305 items
con = duckdb.connect(database=db_string)           
count = con.execute("SELECT COUNT(*) FROM W2V_PROTEIN_UREF100_E").fetchall()
print(count[0])
con.close()

(95272305,)


In [ ]:
# create an index (after loading the data) - initially ran out of memory on Macbook with all proteins
# Took 25s with eukaryotic only
#
# Going to create 2 indices as have added a counter column and want an index
#
con = duckdb.connect(database=db_string)   
con.execute("CREATE INDEX UNIREF100_IDX ON W2V_PROTEIN_UREF100_E(UNIPROT_ID)")
con.execute("CREATE INDEX COUNTER_IDX ON W2V_PROTEIN_UREF100_E(COUNTER)")
print('indices created')
con.close()

In [18]:
con = duckdb.connect(database=db_string)   
results = con.execute("SELECT * FROM W2V_PROTEIN_UREF100_E WHERE COUNTER >= 30000000 AND COUNTER <30000100 ").fetchall()
for res in results:
    print (res)
con.close()

(30000000, 'A0A6M2CIG0', 372, 1, 373, 2, 6941, 'Rhipicephalus microplus')
(30000001, 'A0A6M2CIG2', 436, 1, 437, 1, 6941, 'Rhipicephalus microplus')
(30000002, 'A0A6M2CIG3', 503, 1, 504, 1, 6941, 'Rhipicephalus microplus')
(30000003, 'A0A6M2CIG5', 512, 1, 513, 1, 6941, 'Rhipicephalus microplus')
(30000004, 'A0A6M2CIG7', 519, 1, 520, 1, 6941, 'Rhipicephalus microplus')
(30000005, 'A0A6M2CIG9', 84, 1, 85, 1, 6941, 'Rhipicephalus microplus')
(30000006, 'A0A6M2CIH1', 518, 1, 519, 1, 6941, 'Rhipicephalus microplus')
(30000007, 'A0A6M2CIH2', 215, 1, 216, 1, 6941, 'Rhipicephalus microplus')
(30000008, 'A0A6M2CIH3', 546, 1, 547, 1, 6941, 'Rhipicephalus microplus')
(30000009, 'A0A6M2CIH5', 204, 1, 205, 1, 6941, 'Rhipicephalus microplus')
(30000010, 'A0A6M2CIH6', 226, 1, 227, 1, 6941, 'Rhipicephalus microplus')
(30000011, 'A0A6M2CIH8', 96, 1, 97, 1, 6941, 'Rhipicephalus microplus')
(30000012, 'A0A6M2CII0', 561, 1, 562, 1, 6941, 'Rhipicephalus microplus')
(30000013, 'A0A6M2CII1', 269, 1, 270, 1, 6

### LOAD PFAM TOKENS INTO W2V_TOKEN

In [14]:
# July 20 2024 - Took 1m 55s to load 296,017,815 entries from a directory on a macbook
# July 31 2024 - Restest took 3m 10s from an external drive attached to macbook
con = duckdb.connect(database=db_string)

con.execute("CREATE TABLE W2V_TOKEN AS SELECT * FROM read_csv_auto('/Volumes/My Passport/data/pfam/protein2ipr_pfam_20240715.dat', columns={'uniprot_id' :'VARCHAR', 'type' : 'VARCHAR', 'token' : 'VARCHAR', 'start': 'USMALLINT', 'end': 'USMALLINT'})")
con.close()

In [34]:
# with pfam only this shows 296,017,815 entries
# after loading disorder as well this shows 377,274,915 (81,257,100 disorder entries)
con = duckdb.connect(database=db_string)           
protein_count = con.execute("SELECT COUNT(*) FROM W2V_TOKEN").fetchall()
print(protein_count)
con.close()

[(377274915,)]


In [16]:
# create an index (after loading data)
con = duckdb.connect(database=db_string)  
res = con.execute("CREATE INDEX PF_TKN_IDX ON W2V_TOKEN(UNIPROT_ID)")
con.close()

In [52]:
# test
con = duckdb.connect(database=db_string)
#token = 'PF19782' # Has 0 eukaryotic proteins
#token = 'PF20176' # Has 0 eukaryotic proteins
token = 'PF20200' # Has 0 eukaryotic proteins
#token = 'PF14033' # Has about 4852 eukaryotic proteins

# get number of times this token is in W2V_TOKEN
token_count = con.execute("SELECT COUNT(*) FROM W2V_TOKEN WHERE TOKEN=(?)", [token]).fetchall()
print(f"found {token_count[0][0]} entries for {token}")

# get the protein id for each token
results = con.execute("SELECT DISTINCT UNIPROT_ID FROM W2V_TOKEN WHERE TOKEN=(?)", [token]).fetchall()

print(f"Found {len(results)} unique proteins containing {token}")


# need to check if that protein is actually eukaryotic
eukaryotic_count = 0
for protein_res in results:
    #print(res)
    protein_id = protein_res[0]
    
    # check for protein_id
    protein_count = con.execute("SELECT COUNT(*) FROM W2V_PROTEIN WHERE UNIPROT_ID=(?)", [protein_id]).fetchall()
    
    count = protein_count[0][0]
    
    if(count >0):
        eukaryotic_count +=1
        
print(f"Found {eukaryotic_count} eukaryotic proteins with pfam entry {token}") 
    
con.close()

found 614 entries for PF20200
Found 607 unique proteins containing PF20200
Found 0 eukaryotic proteins with pfam entry PF20200


 ### LOAD DISORDER ITEMS INTO W2V_TOKEN

In [18]:
# Load disorder entries
# First run : July 19
# Retest    : August 1st (on Macbook - took 2mn 25s)
con = duckdb.connect(database=db_string) 
con.execute("INSERT INTO W2V_TOKEN SELECT * FROM read_csv_auto('/Volumes/My Passport/data/disorder/dat/disordered_tokens_20240719.dat')")
con.close()

In [20]:
# with pfam only this shows 296,017,815 entries
# after loading disorder as well this shows 377,274,915
con = duckdb.connect(database=db_string)           
token_count = con.execute("SELECT COUNT(*) FROM W2V_TOKEN").fetchall()
print(token_count)
con.close()

[(377274915,)]


In [22]:
# test that W2V_TOKEN has all pfam and disorder entries
con = duckdb.connect(database=db_string)           
tokens = con.execute("SELECT * FROM W2V_TOKEN WHERE UNIPROT_ID=(?)", ['A0A010PZU8']).fetchall()
print(tokens)
con.close()

[('A0A010PZU8', 'PFAM', 'PF00400', 865, 900), ('A0A010PZU8', 'PFAM', 'PF00400', 928, 955), ('A0A010PZU8', 'PFAM', 'PF00400', 960, 998), ('A0A010PZU8', 'PFAM', 'PF00400', 1017, 1040), ('A0A010PZU8', 'PFAM', 'PF00400', 1078, 1108), ('A0A010PZU8', 'PFAM', 'PF00400', 1233, 1260), ('A0A010PZU8', 'PFAM', 'PF05729', 358, 479), ('A0A010PZU8', 'PFAM', 'PF17100', 152, 254), ('A0A010PZU8', 'DISORDER', 'Consensus Disorder Prediction', 1, 30)]


### LOAD TAXONOMY INFO

#### Names

In [4]:
# see the data-preparation folder for a shell script that produces the .dat file loaded here
con = duckdb.connect(database=db_string)
con.execute("CREATE TABLE W2V_TAX_NAME AS SELECT * FROM read_csv_auto('/Volumes/My Passport/data/taxonomy/dat/scientific_names_20240802.dat', columns={'tax_id' :'VARCHAR', 'name' : 'VARCHAR'})")
con.close()

In [5]:
# count  - should have 2,588,170 entries
con = duckdb.connect(database=db_string)           
token_count = con.execute("SELECT COUNT(*) FROM W2V_TAX_NAME").fetchall()
print(token_count)
con.close()

[(2588170,)]


In [10]:
# create an index (after loading data)
con = duckdb.connect(database=db_string)  
res = con.execute("CREATE INDEX TAX_NM_IDX ON W2V_TAX_NAME(TAX_ID)")
con.close()

#### Categories

In [7]:
# see the data-preparation folder for a shell script that produces the .dat file loaded here
con = duckdb.connect(database=db_string)
con.execute("CREATE TABLE W2V_TAX_CAT AS SELECT * FROM read_csv_auto('/Volumes/My Passport/data/taxonomy/dat/categories_20240802.dat', columns={'type' : 'VARCHAR', 'parent_id' :'VARCHAR', 'id' : 'VARCHAR'})")
con.close()

In [9]:
# count  - should have 1,567,316 entries
con = duckdb.connect(database=db_string)           
token_count = con.execute("SELECT COUNT(*) FROM W2V_TAX_CAT").fetchall()
print(token_count)
con.close()

[(1567316,)]


In [11]:
# create an index (after loading data)
con = duckdb.connect(database=db_string)  
res = con.execute("CREATE INDEX TAX_CT_IDX ON W2V_TAX_CAT(ID)")
con.close()

In [2]:
# test that W2V_TOKEN has all pfam and disorder entries
# 1445577   : Colletotrichum fioriniae PJ7
# 10116     : Rattus norvegicus
con = duckdb.connect(database=db_string)           
tokens = con.execute("SELECT * FROM W2V_TAX_NAME WHERE TAX_ID=(?)", ['1310608']).fetchall()
print(tokens)
con.close()

[('1310608', 'Acinetobacter sp. 1295259')]


## ENCODING AND ANALYSIS

#### FIND UNIQUE PFAM FOR EUKARYOTIC

In [23]:

output_file_root = "/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/tmp/eukaryotic_pfam_smart_not_unique_"

#
# IMPORTANT - WHY WE NEED THIS
#
# W2V_TOKEN was created from protein2ipr.dat and thus has loads of non-eukaryiotic entries
# that are not relevant. This script creates a list of pfam entries for eukaryotic proteins 
# only by joining across W2V_PROTEIN_UREF100_E and W2V_TOKEN by UNIPROT_ID
#
# W2V_PROTEIN_UREF100_E only has eukaryotic proteins in it
#
# This script outputs to a number of files all protein-pfam couples, there are many repeated elements
# A new file is created for each 'chunk'.These can be combined into one and then sorted into a unique
# list - see 'combine_pfam_list.sh'
#
#
# *** VERY IMPORTANT ****
# Having a counter on W2V_PROTEIN_UREF100_E and moving through that table in chunks
# is SIGNIFICANTLY faster that doing a join using LIMIT and OFFSET
#
def get_eukaryotic_pfams(start_pos, end_pos, iteration):
    #print(f"iteration {iteration} from {start_pos} to {end_pos}.")
    s = time.time()
    
    output_file = output_file_root+ str(iteration) + ".txt"
    # create long life/expensive objects
    of  = open(output_file, "w")
    con = duckdb.connect(database=db_string)
    
    try:
        #con = duckdb.connect(database=db_string)  
        #results = con.execute(f"SELECT W2V_PROTEIN_UREF100_E.UNIPROT_ID, W2V_TOKEN.TOKEN FROM ( SELECT UNIPROT_ID FROM W2V_PROTEIN_UREF100_E ORDER BY UNIPROT_ID LIMIT {limit} OFFSET {offset}) AS W2V_PROTEIN_UREF100_E INNER JOIN W2V_TOKEN AS W2V_TOKEN ON W2V_PROTEIN_UREF100_E.UNIPROT_ID = W2V_TOKEN.UNIPROT_ID WHERE W2V_TOKEN.TYPE = 'PFAM' ").fetchall()
        results = con.execute(f"SELECT W2V_PROTEIN_UREF100_E.UNIPROT_ID, W2V_TOKEN.TOKEN FROM ( SELECT UNIPROT_ID FROM W2V_PROTEIN_UREF100_E WHERE COUNTER >= {start_pos} and COUNTER < {end_pos}) AS W2V_PROTEIN_UREF100_E INNER JOIN W2V_TOKEN AS W2V_TOKEN ON W2V_PROTEIN_UREF100_E.UNIPROT_ID = W2V_TOKEN.UNIPROT_ID WHERE W2V_TOKEN.TYPE = 'PFAM' ").fetchall()
    except Exception as e:
        print(f"Error on iteration {iteration}, {e}, closing finr {output_file}")
        of.close()
        con.close()
        return
    e1 = time.time()

    for res in results:
        #print(res[1])
        of.write(res[1] +'\n')        
    e2 = time.time()

    print(f"iteration {iteration} from {start_pos} to {end_pos}. query took {e1-s}s, overall took {e2-s}s")

    of.close()
    con.close()

num_eukaryotic = 95272305
start_pos       = 0    # start point
chunk_size      = 500000    # how many rows to return
end_pos         = chunk_size
iterations      = (num_eukaryotic // chunk_size) + 1

# not starting from 0 as alredy did 60
#start_iteration = iterations - 61

print(iterations, 'required.')

for i in range(iterations):
    get_eukaryotic_pfams(start_pos, end_pos, i)
    start_pos += chunk_size
    end_pos += chunk_size


191 required.
iteration 0 from 0 to 500000. query took 5.040165185928345s, overall took 5.111280202865601s
iteration 1 from 500000 to 1000000. query took 5.5530312061309814s, overall took 5.624272108078003s
iteration 2 from 1000000 to 1500000. query took 5.292984962463379s, overall took 5.377043962478638s
iteration 3 from 1500000 to 2000000. query took 5.45654821395874s, overall took 5.533218145370483s
iteration 4 from 2000000 to 2500000. query took 5.277507781982422s, overall took 5.312421798706055s
iteration 5 from 2500000 to 3000000. query took 5.170738220214844s, overall took 5.219146013259888s
iteration 6 from 3000000 to 3500000. query took 5.81486177444458s, overall took 5.8929338455200195s
iteration 7 from 3500000 to 4000000. query took 5.861753940582275s, overall took 5.918999910354614s
iteration 8 from 4000000 to 4500000. query took 5.312227725982666s, overall took 5.3758320808410645s
iteration 9 from 4500000 to 5000000. query took 5.828030109405518s, overall took 5.8947670459

### Create table of pfam ids without the 'PF' in front!

In [25]:
con = duckdb.connect(database=db_string)

output_file = "/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/unique_eukaryotic_pfam_ids.txt"          
con.execute("CREATE TABLE W2V_PFAM_E AS SELECT * FROM read_csv_auto('/Users/patrick/dev/ucl/comp0158_mscproject/data/pfam/unique_eukaryotic_pfam_ids.txt', columns={'COUNTER' :'USMALLINT', 'STRIPPED_PFAM_ID': 'USMALLINT', 'PFAM_ID': 'VARCHAR'})")
con.close()

In [26]:
con = duckdb.connect(database=db_string)           
count = con.execute("SELECT COUNT(*) FROM W2V_PFAM_E").fetchall()
print(count[0])
con.close()

(15577,)


: 

## UTILITIES

#### Search for PFAM and PROTEIN ENTRIES

In [26]:
# test that W2V_TOKEN has all pfam and disorder entries
# 1445577   : Colletotrichum fioriniae PJ7
# 10116     : Rattus norvegicus
con = duckdb.connect(database=db_string)

# 1. Test - find a protein with pfam entries
#    - Both of these work
#protein_id = "A0A009GYB3" # this is prob not eukaryotic
protein_id = "A0A010PZJ8"

#tokens = con.execute("SELECT * FROM W2V_TOKEN WHERE UNIPROT_ID = 'A0A009GYB3'").fetchall()
#tokens = con.execute("SELECT * FROM W2V_TOKEN WHERE UNIPROT_ID = (?)", [protein_id] ).fetchall()

# 2. Find that same protein in W2V_PROTEIN
# doesn't work - possibly because the pfam entries are from all proteins whereas W2V_PROTEIN only
# has TrEMBL Eukaryotic proteins
#tokens = con.execute("SELECT * FROM W2V_PROTEIN WHERE UNIPROT_ID = 'A0A009GYB3'").fetchall()
tokens = con.execute("SELECT * FROM W2V_PROTEIN WHERE UNIPROT_ID = (?)", [protein_id]).fetchall()
print('W2V_PROTEIN', tokens)

# doesn't work
#tokens = con.execute("SELECT * FROM W2V_TOKEN WHERE UNIPROT_ID = (?)", ['protein_id']).fetchall()

# none of these work - is the protein A0A009GYB3 in UniRef??
# tokens = con.execute("SELECT * FROM W2V_PROTEIN_UNIREF_100_ALL_TAX WHERE UNIPROT_ID = 'A0A009GYB3'").fetchall()
tokens = con.execute("SELECT * FROM W2V_PROTEIN_UREF100_E WHERE UNIPROT_ID = (?)", [protein_id]).fetchall()
# tokens = con.execute("SELECT * FROM W2V_PROTEIN_UNIREF_100_ALL_TAX WHERE UNIPROT_ID = (?)", [protein_id]).fetchall()
# grep "A0A009GYB3" uniref100_tax_20240801.dat > returns nothing

print('W2V_PROTEIN_UREF100_E', tokens)
con.close()

W2V_PROTEIN [('A0A010PZJ8', 1, 494)]
W2V_PROTEIN_UREF100_E [('UniRef100', 'A0A010PZJ8', 493, 1, 494, 1, 1445577, 'Colletotrichum fioriniae PJ7')]


In [40]:
# test that W2V_TOKEN has all pfam and disorder entries
# 1445577   : Colletotrichum fioriniae PJ7
# 10116     : Rattus norvegicus
con = duckdb.connect(database=db_string)           
tokens = con.execute("SELECT * FROM W2V_TAX_CAT WHERE ID=(?)", ['1445577']).fetchall()
print(tokens)
tokens = con.execute("SELECT * FROM W2V_TAX_NAME WHERE TAX_ID=(?)", ['1445577']).fetchall()
print(tokens)
con.close()

[('E', '710243', '1445577')]
[('1445577', 'Colletotrichum fioriniae PJ7')]


#### Drop Table

In [16]:
con = duckdb.connect(database=db_string)           
con.execute("DROP TABLE PROTEIN")
con.close()

#### Unlock database

In [21]:
import duckdb
import os

# this doesn;t seem to work....
def is_locked():
    lock_file = f'{db_string}.lock'
    return os.path.exists(lock_file)

is_locked()

# ... but this does from a command prompt
#fuser database/proteins.db

fuser /Users/patrick/dev/ucl/comp0158_mscproject/database/w2v_20240731_test.db

# then kill -9 <id if there is one list>

False

: 